In [1]:
import os
import numpy as np
import pandas as pd

from collections import defaultdict
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

In [2]:
def get_n_m_combi():
    return [(0, 0)] + [(n, m) for m in range(5, 16, 2) for n in range(1, 4)]

def get_paths(n, m):
    return [r"D:\Downloads\checkpoints\checkpoint_hyp\checkpoint_N_{}_M_{}\CompCars\fold_{}\log".format(n, m, i) for i in range(1, 6)]

In [3]:
def tabulate_events(dpath):
    final_out = {}
    for dname in os.listdir(dpath):
        ea = EventAccumulator(os.path.join(dpath, dname)).Reload()
        tags = ea.Tags()['scalars']
        out = {}
        for tag in tags:
            tag_values=[]
            wall_time=[]
            steps=[]
            for event in ea.Scalars(tag):
                tag_values.append(event.value)
                wall_time.append(event.wall_time)
                steps.append(event.step)
            out[tag]=pd.DataFrame(data=dict(zip(steps,np.array([tag_values,wall_time]).transpose())), columns=steps,index=['value','wall_time'])
        if len(tags)>0:      
            df= pd.concat(out.values(),keys=out.keys())
            final_out[dname] = df
    return final_out

In [5]:
res = pd.DataFrame(columns=["n", "m", "avg_min_loss", "loss_std", "avg_max_acc", "acc_std"])
n_m_combi = get_n_m_combi()
for n, m in n_m_combi:
    min_losses = []
    max_accs = []
    for path in get_paths(n, m):
        df_per_epoch = list(tabulate_events(path).values())
        min_losses.append(min([float(df.loc[("Test/total_loss_avg", "value")]) for df in df_per_epoch]))
        max_accs.append(max([float(df.loc[("Test/local_accuracy_1", "value")]) for df in df_per_epoch]))
    res.loc[len(res)] = [n, m, np.mean(min_losses), np.std(min_losses), np.mean(max_accs), np.std(max_accs)]
    print(f"n={n}, m={m} done")

n=0, m=0 done
n=1, m=5 done
n=2, m=5 done
n=3, m=5 done
n=1, m=7 done
n=2, m=7 done
n=3, m=7 done
n=1, m=9 done
n=2, m=9 done
n=3, m=9 done
n=1, m=11 done
n=2, m=11 done
n=3, m=11 done
n=1, m=13 done
n=2, m=13 done
n=3, m=13 done
n=1, m=15 done
n=2, m=15 done
n=3, m=15 done


In [6]:
res.to_csv("randaug.csv", index=False)

In [7]:
res

,n,m,avg_min_loss,loss_std,avg_max_acc,acc_std
0,0.0,0.0,10.758668,0.047249,0.955660,0.008429
1,1.0,5.0,10.755056,0.083395,0.960662,0.012843
2,2.0,5.0,10.731104,0.066317,0.963773,0.007548
3,3.0,5.0,10.755368,0.096247,0.961277,0.014595
4,1.0,7.0,10.749939,0.046314,0.961900,0.007748
5,2.0,7.0,10.742721,0.045069,0.959410,0.010024
6,3.0,7.0,10.714740,0.075218,0.961914,0.013823
7,1.0,9.0,10.722394,0.065316,0.966900,0.008502
8,2.0,9.0,10.736215,0.059888,0.963152,0.013155
9,3.0,9.0,10.741429,0.066535,0.958783,0.006614
